In [1]:
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
import numpy as np
import os

In [5]:
def display_predictions(images, predictions, ground_truths):
    for img, pred, gt in zip(images, predictions, ground_truths):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for plotting
        
        # Draw ground truth boxes
        for gt_box in gt:
            x1, y1, x2, y2, cls = gt_box
            img = cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            img = cv2.putText(img, f'{cls}', (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Draw predicted boxes
        for pred_box in pred:
            x1, y1, x2, y2, conf, cls = pred_box
            img = cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            img = cv2.putText(img, f'{cls} {conf:.2f}', (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.axis('off')
        plt.show()

def test_yolo(model_path, images_path, labels_path, img_size=640):
    # Load the model
    model = YOLO(model_path)

    # Load test images and annotations
    images = []  # List to store test images
    ground_truths = []  # List to store ground truth boxes

    for img_file in os.listdir(images_path):
        if img_file.endswith('.jpg'):
            img_path = os.path.join(images_path, img_file)
            img = cv2.imread(img_path)
            images.append(img)

            # Load ground truth annotations (assuming they are in YOLO format)
            annot_file = img_file.replace('.jpg', '.txt')
            annot_path = os.path.join(labels_path, annot_file)
            gt_boxes = []
            with open(annot_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    cls = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:])
                    x1 = (x_center - width / 2) * img.shape[1]
                    y1 = (y_center - height / 2) * img.shape[0]
                    x2 = (x_center + width / 2) * img.shape[1]
                    y2 = (y_center + height / 2) * img.shape[0]
                    gt_boxes.append((x1, y1, x2, y2, cls))
            ground_truths.append(gt_boxes)

    # Run predictions
    predictions = []
    for img in images:
        results = model.predict(img, imgsz=img_size)
        pred_boxes = []
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy
                conf = box.conf
                cls = box.cls
                pred_boxes.append((x1.item(), y1.item(), x2.item(), y2.item(), conf.item(), int(cls.item())))
        predictions.append(pred_boxes)

    # Display predictions and ground truths
    display_predictions(images, predictions, ground_truths)




# Replace with actual paths
test_yolo('~/shared/datasets_for_training/YOLO_military_Trucks/runs/detect/train2/weights/best.pt',
          '~/shared/datasets_for_training/YOLO_military_Truck/test/images',
          '~/shared/datasets_for_training/YOLO_military_Truck/test/labels')

FileNotFoundError: [Errno 2] No such file or directory: '~/shared/datasets_for_training/YOLO_military_Trucks/runs/detect/train2/weights/best.pt'

In [8]:
! cd '~/shared'

/bin/bash: line 1: cd: ~/shared: No such file or directory
